In [14]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd 
import dotenv
import os
import psycopg2

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASSWORD,
    port=PG_PORT,
    sslmode="require",
)

data = pd.read_sql("""WITH transition_data AS (
    SELECT
		spadl.game_id,
		spadl.period_id,
        spadl.player_id,
        spadl.team_id,
        spadl.start_x,
        spadl.start_y,
        spadl.end_x,
        spadl.end_y,
        spadl.seconds,
        spadl.action_type,
        spadl.result,
        spadl.bodypart,
        m.home_score,
        m.away_score,
        CASE
            WHEN spadl.team_id = m.home_team_id THEN m.home_score
            ELSE m.away_score
        END AS score_team,
        
        CASE
            WHEN spadl.team_id = m.home_team_id THEN m.away_score
            ELSE m.home_score
        END AS score_opponent,
        
        SQRT(POWER(spadl.end_x - spadl.start_x, 2) + POWER(spadl.end_y - spadl.start_y, 2)) / NULLIF(spadl.seconds, 0) AS transition_speed,
        t1.team_name AS team_name,
        t2.team_name AS opponent_team_name
    FROM
        spadl_actions spadl
    JOIN teams t1 ON spadl.team_id = t1.team_id  
    JOIN teams t2 ON spadl.team_id != t2.team_id  
    JOIN matches m ON spadl.game_id = m.match_id  
)
SELECT
    game_id,
    period_id,
    player_id,
    team_id,
    team_name,
    opponent_team_name,
    start_x,
    start_y,
    end_x,
    end_y,
    seconds,
    action_type,
    result,
    transition_speed,
    score_team,
    score_opponent,
    CASE
        WHEN transition_speed > 10 THEN 'fast'
        ELSE 'slow'
    END AS transition_type
FROM
    transition_data
ORDER BY
    game_id, period_id, seconds
limit 500000;""", conn)





C:\Users\axlnu\AppData\Local\Temp\ipykernel_18244\1621837446.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("""WITH transition_data AS (


Index(['game_id', 'period_id', 'player_id', 'team_id', 'team_name',
       'opponent_team_name', 'start_x', 'start_y', 'end_x', 'end_y', 'seconds',
       'action_type', 'result', 'transition_speed', 'score_team',
       'score_opponent', 'transition_type'],
      dtype='object')


KeyError: "['transition_type'] not found in axis"